In [ ]:
import sklearn
import pandas as pd
import numpy as np
import nltk
import sklearn
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import matplotlib
from sklearn.model_selection import cross_val_score
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn.model_selection import train_test_split
import re
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
import pickle
from wordcloud import WordCloud

In [ ]:
full_labels = pd.read_csv('../input_data/full_datasets/fully_labelled_texts.csv')

In [ ]:
stemmer = SnowballStemmer("dutch")
np.random.seed(500)

def create_preprocess_corpus(input_corpus, content_name, label_name, smoking=False, second_experiment=False, skip_rows=[]):
    corpus = input_corpus[['Unnamed: 0', content_name, label_name]]
    corpus = corpus.rename({content_name:'text', label_name: 'label'}, axis=1)
    corpus['processed_text'] = corpus['text'].str.replace('\t',' ')
    corpus.drop_duplicates(inplace=True)
    corpus['processed_text'] = corpus['processed_text'].astype(str)
    corpus['processed_text'] = corpus['processed_text'].str.lower()
    corpus['processed_text'] = [stemmer.stem(text) for text in corpus['processed_text']]
    if second_experiment:
        if smoking:
            replace_text = 'Rookt niet'
            corpus['label'] = corpus['label'].str.replace('Niets gevonden', 'Rookt niet')
            corpus['label'] = corpus['label'].str.replace('Rookte', 'Rookt niet')
        else:
            corpus['label'] = corpus['label'].str.replace('Niets gevonden', 'Nee')
        
    corpus = corpus.drop(corpus[corpus.label == '--'].index)
    corpus = corpus.drop(corpus[corpus.label == 'Onbekend'].index)
    corpus = corpus[~corpus.processed_text.str.contains('vertrouwelijk')]
    corpus_backup = corpus.copy()
    return corpus, corpus_backup

def add_processed_text(input_corpus, text_column):
    corpus = input_corpus.copy()
    corpus['processed_text'] = corpus[text_column].str.replace('\t',' ')
    corpus.drop_duplicates(inplace=True)
    corpus['processed_text'] = corpus['processed_text'].astype(str)
    corpus['processed_text'] = corpus['processed_text'].str.lower()
    corpus['processed_text'] = [stemmer.stem(text) for text in corpus['processed_text']]
    return corpus

In [ ]:
existing_labels = pd.read_csv('full_texts_to_label_full4.csv')

In [ ]:
existing_random_labels = pd.read_csv('full_texts_to_label_random1.csv')

In [ ]:
existing_random_labels2 = pd.read_csv('full_texts_to_label_random2.csv')

In [ ]:
full_labels_no_dup = full_labels[~full_labels.content.isin(existing_random_labels.text)][~full_labels.content.isin(existing_random_labels2.text)][~full_labels.content.isin(existing_labels.text)]

In [ ]:
full_labels_no_dup

In [ ]:
random_samples = full_labels_no_dup.sample(frac=1).head(501)

In [ ]:
random_samples

In [ ]:
full_samples = add_processed_text(random_samples, 'content')

In [ ]:
full_texts_to_label = full_samples.rename(columns={'content':'text'})[['text', 'roken_answer_label', 'alcohol_answer_label', 'drugs_answer_label']]

In [ ]:
full_texts_to_label.to_csv('full_texts_to_label_random3.csv')